# LU decomposition

System of linear equations can be solved in different methods. One of the way is through row reduced echelon form (RREF) method. Throughout the time, scientists & researchers have try to come up with more efficient ways to find solutions on this system. Why? It is because in reality, we will need to deal with a huge matrices. And solving it using the RREF method can take quite a time and computing power. As a remedy to this complication, matrix decomposition was introduced. If you have a chance to take machine learning courses, maybe you will stumble upon the Single Value Decomposition concept. This is an example of matrix decomposition.

For today's simulation, we'll walkthrough over other type of decomposition called the LU decomposition. There are many techniques how to split our matrices into lower triangular matrix and upper triangular matrix but we will use video below as our reference:
* https://www.youtube.com/watch?v=jbeX2HCW6OE

In [1]:
import numpy as np

# input is nxn matrix
def LU(matrix):
    dim = len(matrix)
    u_mat = np.array(matrix)
    l_mat = initialL(dim)
    
    for row in np.arange(1,dim):
        for col in np.arange(0,row):
            c = float(u_mat[row][col]) / float(u_mat[col][col])
            l_mat[row][col] = c
            u_mat[row] = u_mat[row] - c*u_mat[col]
    
    return l_mat, u_mat

def P(matrix):
    p_mat = initialL(len(matrix))
    p_mat = p_mat.tolist()
    for col in range(len(matrix)):
        r = highRow(matrix,col)
        p_mat = chgRow(p_mat, col, r)
        matrix = chgRow(matrix, col, r)
       
    return np.array(matrix), np.array(p_mat)

def highRow(matrix, col=1):
    row = len(matrix)
    maxR = np.zeros(row)
    for i in range(row):
        maxR[i] = matrix[i][col]
    return np.argmax(maxR)

def chgRow(matrix, target, replacement):
    target_row = matrix[target]
    replacement_row = matrix[replacement]
    matrix[target] = replacement_row
    matrix[replacement] = target_row
    
    return matrix

def initialL(dim):
    l = np.zeros((dim,dim))
    for row in range(dim):
        for col in range(dim):
            if row == col:
                l[row][col] = 1.0
    return l

def forward_sub(matrix, b):
    dim = len(matrix)
    for row in range(dim):
        for col in range(dim):
            if row == col:
                b[row] = b[row] / float(matrix[row][col])
            else:
                b[row] -= b[col]*float(matrix[row][col])
    return b

def backward_sub(matrix, b):
    dim = len(matrix) - 1
    for row in np.arange(dim,-1,-1):
        for col in np.arange(dim,-1,-1):
            if row == col:
                b[row] = b[row] / float(matrix[row][col])
            else:
                b[row] -= b[col]*float(matrix[row][col])
    return b

In [2]:
# testing:
# x + y + z = 1, 4x + 3y - z = 6, 3x + 5y + 3z = 4
# [[1,1,1],
#  [4,3,-1],
#  [3,5,3]]

# output:
# L = [[1,0,0],      U = [[1,1,1],
#      [4,1,0],          [0,-1,-5],
#      [3,-2,1]]         [0,0,-10]

In [3]:
import numpy as np
A = [[1,1,1],[4,3,-1],[3,5,3]]
l,u = LU(A)
l,u

(array([[ 1.,  0.,  0.],
        [ 4.,  1.,  0.],
        [ 3., -2.,  1.]]),
 array([[  1,   1,   1],
        [  0,  -1,  -5],
        [  0,   0, -10]]))

### Application
Our system of linear equation is going to be as follows:
$$ Ax = b $$

Substitute A=LU

$$ (LU)x = b $$

Applying Associative property for matrix,

$$ L(Ux) = b $$
$$ Lc = b $$

So, to solve for x, we'll reverse the 2 steps above.

In [4]:
forward_sub(l,[1,6,4])

[1.0, 2.0, 5.0]

In [5]:
backward_sub(u, [1,2,5])

[1.0, 0.5, -0.5]

Hence x = [1, 0.5, -0.5]

In [6]:
# output should be [3,1,-2]. we'll arrive to the same answer 
# eventhough we don't get the rref.
# output should be: [1,0, 0, 3] 
#                   [0,1, 0, 1] 
#                   [0,0, 1,-2]

# you can refer to my note called Gauss Jordan Elimination.ipynb
A = [[1,3,2],
    [2,7,7],
    [2,5,2]]

l, u = LU(A)
l, u

(array([[ 1.,  0.,  0.],
        [ 2.,  1.,  0.],
        [ 2., -1.,  1.]]),
 array([[1, 3, 2],
        [0, 1, 3],
        [0, 0, 1]]))

In [7]:
backward_sub(u, forward_sub(l,[2,-1,7]))

[3.0, 1.0, -2.0]

Similar result with our RREF method.

### PLU decomposition

The LU technique above is great if we do not encounter any 0 at our diagonal pivot. However, in case we encounter with such situation, some extra steps need to be done first, which is, we need to permute the row with row that have non-zero value at our pivot point. Let us consider the example below.

In [15]:
# testing:
#       2y + z = 4
#        x + y + 2z = 6
#        2x + y + z = 7
# output should be: [[1. , 0. , 0. , 2.2],
#                    [0. , 1. , 0. , 1.4],
#                    [0. , 0. , 1. , 1.2]]

A = [[0.,2.,1.],
    [1.,1.,2.],
    [2.,1.,1.]]

Here, we can see that the first row have 0 at the first column. We need to exchange this row with other row that has non-zero value for its first column. To be save, we are going to arrange this matrix such that for every column, the pivot point is the largest value.

In [16]:
res, p = P(A)
res, p

(array([[2., 1., 1.],
        [0., 2., 1.],
        [1., 1., 2.]]),
 array([[0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.]]))

Using this permuted matrix, we will now able to decompose it into LU.
Our system of linear equation is going to be as follows:
$$ Ax = b $$

A cannot be decomposed because pivot point have 0.

turn A into a matrix that can be decomposed

$$ PAx = Pb $$

Now, PA=LU,

$$ (LU)x = Pb $$
$$ L(Ux) = Pb $$
$$ Lc = Pb $$


In [17]:
l, u = LU(res)
x = backward_sub(u, forward_sub(l,np.dot(p,[4,6,7]).tolist()))
x

[2.2, 1.4, 1.2]

In [18]:
np.dot(A,x)

array([7., 4., 6.])

And there you go, we got a similar result with our rref method